### SNN - Node classsification

In [ ]:
import os

# set the current working directory
curr_path = os.getcwd().split("/")[:-1]
curr_path = "/".join(curr_path)
os.chdir(curr_path)
os.getcwd()

In [1]:
import numpy as np
import torch

import toponetx.datasets.graph as graph
from topomodelx.utils.sparse import from_sparse

from pytspl.snn import SCCNNTrainer
from pytspl.snn import SCNNTrainer

#### Load dataset

In [2]:
# load dataset
dataset = graph.karate_club(complex_type="simplicial")
print(dataset)

# get incidence matrices
incidence_1 = dataset.incidence_matrix(rank=1)
incidence_2 = dataset.incidence_matrix(rank=2)
print(f"The incidence matrix B1 has shape: {incidence_1.shape}.")
print(f"The incidence matrix B2 has shape: {incidence_2.shape}.")

incidence_1 = from_sparse(incidence_1)
incidence_2 = from_sparse(incidence_2)

incidence_all = (incidence_1, incidence_2)

# get laplacians
laplacian_0 = dataset.hodge_laplacian_matrix(rank=0)
laplacian_down_1 = dataset.down_laplacian_matrix(rank=1)
laplacian_up_1 = dataset.up_laplacian_matrix(rank=1)
laplacian_down_2 = dataset.down_laplacian_matrix(rank=2)
laplacian_up_2 = dataset.up_laplacian_matrix(rank=2)

laplacian_0 = from_sparse(laplacian_0)
laplacian_down_1 = from_sparse(laplacian_down_1)
laplacian_up_1 = from_sparse(laplacian_up_1)
laplacian_down_2 = from_sparse(laplacian_down_2)
laplacian_up_2 = from_sparse(laplacian_up_2)

laplacian_all = (
    laplacian_0,
    laplacian_down_1,
    laplacian_up_1,
    laplacian_down_2,
    laplacian_up_2,
)

Simplicial Complex with shape (34, 78, 45, 11, 2) and dimension 4
The incidence matrix B1 has shape: (34, 78).
The incidence matrix B2 has shape: (78, 45).


In [3]:
y = np.array(
    [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        0,
        1,
        1,
        1,
        1,
        0,
        0,
        1,
        1,
        0,
        1,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
    ]
)

y_true = np.zeros((34, 2))
y_true[:, 0] = y
y_true[:, 1] = 1 - y

y_train = y_true[:30]
y_test = y_true[-4:]
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [4]:
def get_simplicial_features(dataset, rank):
    if rank == 0:
        which_feat = "node_feat"
    elif rank == 1:
        which_feat = "edge_feat"
    elif rank == 2:
        which_feat = "face_feat"
    else:
        raise ValueError(
            "input dimension must be 0, 1 or 2, because features are supported on" +
            "nodes, edges and faces"
        )

    x = list(dataset.get_simplex_attributes(which_feat).values())
    return torch.tensor(np.stack(x))

### SCCNN 

In [5]:
# get simplicial features
x_0 = get_simplicial_features(dataset, rank=0)
x_1 = get_simplicial_features(dataset, rank=1)
x_2 = get_simplicial_features(dataset, rank=2)

feats_all = (x_0, x_1, x_2)

print(f"There are {x_0.shape[0]} nodes with features of imension {x_0.shape[1]}.")
print(f"There are {x_1.shape[0]} edges with features of dimension {x_1.shape[1]}.")
print(f"There are {x_2.shape[0]} faces with features of dimension {x_2.shape[1]}.")

There are 34 nodes with features of imension 2.
There are 78 edges with features of dimension 2.
There are 45 faces with features of dimension 2.


In [6]:
max_rank = dataset.dim

conv_order = 2
in_channels_all = (x_0.shape[-1], x_1.shape[-1], x_2.shape[-1])
intermediate_channels_all = (16, 16, 16)
num_layers = 2
out_channels = 2  # num classes

model = SCCNNTrainer(
    in_channels_all=in_channels_all,
    hidden_channels_all=intermediate_channels_all,
    out_channels=out_channels,
    conv_order=conv_order,
    max_rank=max_rank,
    update_func="sigmoid",
    n_layers=num_layers,
)

In [7]:
optimizer = torch.optim.Adam(model.parameters, lr=0.1)

model.train(
    feats=feats_all,
    incidence_mats=incidence_all,
    laplacians=laplacian_all,
    y_train=y_train,
    optimizer=optimizer,
    num_epochs=100,
)

Epoch: 1 loss: 0.7105 Train_acc: 0.2000
Epoch: 2 loss: 0.6607 Train_acc: 0.6000
Epoch: 3 loss: 0.5933 Train_acc: 0.8333
Epoch: 4 loss: 0.5920 Train_acc: 0.6000
Epoch: 5 loss: 0.5310 Train_acc: 0.8667
Epoch: 6 loss: 0.5275 Train_acc: 0.8333
Epoch: 7 loss: 0.5066 Train_acc: 0.8000
Epoch: 8 loss: 0.5004 Train_acc: 0.8000
Epoch: 9 loss: 0.5004 Train_acc: 0.8333
Epoch: 10 loss: 0.4986 Train_acc: 0.8000
Epoch: 11 loss: 0.4977 Train_acc: 0.8000
Epoch: 12 loss: 0.4971 Train_acc: 0.8000
Epoch: 13 loss: 0.4965 Train_acc: 0.8000
Epoch: 14 loss: 0.4959 Train_acc: 0.8000
Epoch: 15 loss: 0.4953 Train_acc: 0.8000
Epoch: 16 loss: 0.4945 Train_acc: 0.8000
Epoch: 17 loss: 0.4935 Train_acc: 0.8333
Epoch: 18 loss: 0.4925 Train_acc: 0.8333
Epoch: 19 loss: 0.4915 Train_acc: 0.8333
Epoch: 20 loss: 0.4904 Train_acc: 0.8333
Epoch: 21 loss: 0.4893 Train_acc: 0.8333
Epoch: 22 loss: 0.4883 Train_acc: 0.8333
Epoch: 23 loss: 0.4874 Train_acc: 0.8333
Epoch: 24 loss: 0.4702 Train_acc: 0.8333
Epoch: 25 loss: 0.4899 Tr

In [8]:
model.predict(
    feats=feats_all,
    incidence_mats=incidence_all,
    laplacians=laplacian_all,
    y_test=y_test,
)

Test_acc: 0.5000


### SCNN

In [9]:
rank = 1  # simplex level
conv_order_down = 2
conv_order_up = 2

x = get_simplicial_features(dataset, rank)
channels_x = x.shape[-1]

hidden_channels = 16
out_channels = 2  # num classes
num_layers = 1

if rank == 0:
    laplacian_down = None
    laplacian_up = laplacian_0  # the graph laplacian
    conv_order_down = 0
elif rank == 1:
    laplacian_down = laplacian_down_1
    laplacian_up = laplacian_up_1
elif rank == 2:
    laplacian_down = laplacian_down_2
    laplacian_up = laplacian_up_2
else:
    raise ValueError("Rank must be not larger than 2 on this dataset")

In [10]:
model = SCNNTrainer(
    in_channels=channels_x,
    hidden_channels=hidden_channels,
    out_channels=out_channels,
    conv_order_down=conv_order_down,
    conv_order_up=conv_order_up,
    n_layers=num_layers,
)

print(model.network)

Network(
  (base_model): SCNN(
    (layers): ModuleList(
      (0): SCNNLayer()
    )
  )
  (linear): Linear(in_features=16, out_features=2, bias=True)
)


In [11]:
optimizer = torch.optim.Adam(model.parameters, lr=0.1)

model.train(
    feats=x,
    incidence_1=incidence_1,
    laplacian_down=laplacian_down,
    laplacian_up=laplacian_up,
    y_train=y_train,
    optimizer=optimizer,
    num_epochs=100,
)

Epoch: 1 loss: 0.7545 Train_acc: 0.3333
Epoch: 2 loss: 0.7243 Train_acc: 0.3333
Epoch: 3 loss: 0.7033 Train_acc: 0.9000
Epoch: 4 loss: 0.6766 Train_acc: 0.8000
Epoch: 5 loss: 0.6520 Train_acc: 0.8000
Epoch: 6 loss: 0.6327 Train_acc: 0.8000
Epoch: 7 loss: 0.6187 Train_acc: 0.8000
Epoch: 8 loss: 0.6083 Train_acc: 0.8333
Epoch: 9 loss: 0.5999 Train_acc: 0.8333
Epoch: 10 loss: 0.5925 Train_acc: 0.8333
Epoch: 11 loss: 0.5850 Train_acc: 0.8333
Epoch: 12 loss: 0.5769 Train_acc: 0.8667
Epoch: 13 loss: 0.5716 Train_acc: 0.8667
Epoch: 14 loss: 0.5697 Train_acc: 0.9000
Epoch: 15 loss: 0.5687 Train_acc: 0.9000
Epoch: 16 loss: 0.5675 Train_acc: 0.9000
Epoch: 17 loss: 0.5659 Train_acc: 0.9000
Epoch: 18 loss: 0.5640 Train_acc: 0.9000
Epoch: 19 loss: 0.5621 Train_acc: 0.9000
Epoch: 20 loss: 0.5603 Train_acc: 0.9000
Epoch: 21 loss: 0.5590 Train_acc: 0.9000
Epoch: 22 loss: 0.5586 Train_acc: 0.9000
Epoch: 23 loss: 0.5593 Train_acc: 0.9000
Epoch: 24 loss: 0.5596 Train_acc: 0.9000
Epoch: 25 loss: 0.5587 Tr

In [12]:
model.predict(
    feats=x,
    incidence_1=incidence_1,
    laplacian_down=laplacian_down,
    laplacian_up=laplacian_up,
    y_test=y_test,
)

Test_acc: 0.7500
